In [1]:
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://archive

In [3]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-29 17:25:44--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-11-29 17:25:44 (6.18 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AMZreviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [25]:
# Read the first TSV.GZ file from an S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_game_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_game_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [29]:
# Count the amount of rows in the dataset
print(f"Rows in dataset: {video_game_data_df.count()}")

Rows in dataset: 1785997


In [30]:
# Count the amount of distinct rows in the dataset to find duplicates
print(f"Distinct rows in dataset: {video_game_data_df.distinct().count()}")

Distinct rows in dataset: 1785997


In [31]:
# Drop unnecessary columns
video_game_data_df = video_game_data_df.drop("marketplace", "product_category", "verified_purchase", "review_headline", "review_body")

video_game_data_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|          5|            0|          0|   N| 2015-08-31|
|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|          5|            0|          0|   N| 2015-08-31|
|    2331478|R3BH071QLH8QMC|B0029CSOD2|      98937668|Hidden Mysteries:...|          1|            0|          1|   N| 2015-08-31|
|   52495923|R127K9NTSXA2YH|B00GOOSV98|      23143350|GelTabz Performan...|          3|            0|          0|   N| 2015-08-31|
|   14533949|R32ZWUXDJPW27Q|B00Y074JOM|     821342511|Zero Suit Samus a...|        

In [32]:
# Find unique customer_id counts
print(f"Unique customer_id counts: {video_game_data_df.select('customer_id').distinct().count()}")

Unique customer_id counts: 1045733


In [33]:
# Find instances of each customer_id
customer_id_count_df = video_game_data_df.groupBy('customer_id').count()

customer_id_count_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   48670265|    1|
|   49103216|    2|
|    1131200|    1|
|   43076447|    2|
|   46261368|    1|
|    4883305|    5|
|   41192649|    1|
|   40985731|    7|
|   10437900|    2|
|   22245671|    1|
|    2574873|    1|
|    4696154|    1|
|    5621202|    1|
|    5871933|    2|
|   44089812|    1|
|    2845910|    1|
|    5274369|    1|
|   39069693|    2|
|     137793|    1|
|   31914942|    3|
+-----------+-----+
only showing top 20 rows



In [34]:
# Join count to original dataframe
video_game_data_df = video_game_data_df.join(customer_id_count_df, video_game_data_df.customer_id == customer_id_count_df.customer_id, "left").drop(video_game_data_df.customer_id)
video_game_data_df.show()

+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+-----+
|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|customer_id|count|
+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+-----+
|R1VTIA3JTYBY02|B00008KTNN|     384411423|Metal Arms: Glitc...|          5|            0|          0|   N| 2015-08-31|     106569|    1|
|R3BH071QLH8QMC|B0029CSOD2|      98937668|Hidden Mysteries:...|          1|            0|          1|   N| 2015-08-31|    2331478|    3|
|R3AQQ4YUKJWBA6|B002UBI6W6|     328764615|Psyclone Recharge...|          1|            0|          0|   N| 2015-08-31|    2377552|    1|
|R23H79DHOZTYAU|B0081EH12M|     770100932|New Trigger Grips...|          1|            1|          1|   N| 2015-08-31|    2963837|    2|
|R3IMF2MQ3OU9ZM|B002I0HIMI|     988218515

In [35]:
# Ensure rows are preserved
print(video_game_data_df.count())

1785997


In [36]:
# Rename count column
video_game_data_df = video_game_data_df.withColumnRenamed("count", "customer_count")
video_game_data_df.show()

+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+--------------+
|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|customer_id|customer_count|
+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+--------------+
|R1VTIA3JTYBY02|B00008KTNN|     384411423|Metal Arms: Glitc...|          5|            0|          0|   N| 2015-08-31|     106569|             1|
|R3BH071QLH8QMC|B0029CSOD2|      98937668|Hidden Mysteries:...|          1|            0|          1|   N| 2015-08-31|    2331478|             3|
|R3AQQ4YUKJWBA6|B002UBI6W6|     328764615|Psyclone Recharge...|          1|            0|          0|   N| 2015-08-31|    2377552|             1|
|R23H79DHOZTYAU|B0081EH12M|     770100932|New Trigger Grips...|          1|            1|          1|   N| 2015-08-31|    29

In [38]:
# Check data types
video_game_data_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = true)



In [39]:
# Cast customer_count to integer
video_game_data_df = video_game_data_df.withColumn("customer_count", video_game_data_df.customer_count.cast("integer"))
video_game_data_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = true)

